In [1]:
import numpy as np

#load ascii text and convert to lowercase
filename = "alice.txt"
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

#create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

#summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print('Total characters: ', n_chars)
print('Total vocab: ', n_vocab)

Total characters:  144583
Total vocab:  50


In [2]:
#prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i+seq_length]
  seq_out = raw_text[i+seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patterns: ", n_patterns)

Total patterns:  144483


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

#reshape X to be [sampe, time steps, features]
X = torch.tensor(dataX, dtype = torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([144483, 100, 1]) torch.Size([144483])


In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

class CharModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm = nn.LSTM(input_size = 1, hidden_size = 256, num_layers = 1, batch_first = True)
    self.dropout = nn.Dropout(0.2)
    self.linear = nn.Linear(256, n_vocab)

  def forward(self, x):
    x, _ = self.lstm(x)
    #take only the last output
    x = x[:,-1,:]
    #produce output
    x=self.linear(self.dropout(x))
    return x

In [5]:
#Calculate loss and formulate best model. The model with the lowest loss will be used for text generation
n_epochs = 20
batch_size = 128
model = CharModel()

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X,y), shuffle=True, batch_size = batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
  model.train()
  for X_batch, y_batch in loader:
    y_pred = model(X_batch)
    loss = loss_fn(y_pred, y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  #validation
  model.eval()
  loss = 0
  with torch.no_grad():
    for X_batch, y_batch in loader:
      y_pred = model(X_batch)
      loss += loss_fn(y_pred, y_batch)
    if loss < best_loss:
      best_loss = loss
      best_model = model.state_dict()
    print("Epoch %d: Cross-entropy: % .4f" % (epoch, loss))

torch.save([best_model, char_to_int], "single-char.pth")

Epoch 0: Cross-entropy:  409744.6562
Epoch 1: Cross-entropy:  389397.0938
Epoch 2: Cross-entropy:  375741.4375
Epoch 3: Cross-entropy:  364444.0938
Epoch 4: Cross-entropy:  354655.0312
Epoch 5: Cross-entropy:  347332.9062
Epoch 6: Cross-entropy:  337877.5000
Epoch 7: Cross-entropy:  329361.4688
Epoch 8: Cross-entropy:  323542.1875
Epoch 9: Cross-entropy:  316755.9375
Epoch 10: Cross-entropy:  310818.9688
Epoch 11: Cross-entropy:  304734.7500
Epoch 12: Cross-entropy:  298894.8125
Epoch 13: Cross-entropy:  292670.8750
Epoch 14: Cross-entropy:  286296.9688
Epoch 15: Cross-entropy:  281316.7188
Epoch 16: Cross-entropy:  276478.2500
Epoch 17: Cross-entropy:  273235.2188
Epoch 18: Cross-entropy:  267958.5312
Epoch 19: Cross-entropy:  263817.1562


In [6]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]

In [9]:
#Basic LSTM to generate text
import numpy as np
import torch
import torch.nn as nn

best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i,c) for c,i in char_to_int.items())

#reload the model
class CharModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
    self.dropout= nn.Dropout(0.2)
    self.linear = nn.Linear(256, n_vocab)
  
  def forward(self, x):
    x, _ = self.lstm(x)
    #take only the last output
    x=x[:,-1,:]
    #produce output
    x=self.linear(self.dropout(x))
    return x

model = CharModel()
model.load_state_dict(best_model)

#randomly generate a prompt
filename = "alice.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt=raw_text[start:start+seq_length]
pattern=[char_to_int[c] for c in prompt]

model.eval()
print("Prompt: %s" % prompt)
with torch.no_grad():
  for i in range(1000):
    #format input array of int into PyTorch tensor
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    x = torch.tensor(x, dtype=torch.float32)
    #generate logits as output from the model
    prediction = model(x)
    #convert logits into one character
    index = int(prediction.argmax())
    result = int_to_char[index]
    print(result, end="")
    #append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern = pattern[1:]
print()
print("Done!")

Prompt: f
_that_ is—‘take care of the sense, and the sounds will take care of
themselves.’”

“how fond she i
ore toine iire t shene dane an ince,” said the macc tfrtree. 
“i toon to then io the sooe,” she macc thehe degan.
“oo you moow that io the doerense tound ”ou solle to toe toinge io the siaee.”

“i whsh i can _oeed ”hu was in ” said the macc tfrtree. 
“it woutd you know that ”ou cane doeng ” said the macc tfrtree. 
“i toon to then io the sooe,” she macc thehe degan.
“oo you moow that io the doerense tound ”ou solle to toe toinge io the siaee.”

“i whsh i can _oeed ”hu was in ” said the macc tfrtree. 
“it woutd you know that ”ou cane doeng ” said the macc tfrtree. 
“i toon to then io the sooe,” she macc thehe degan.
“oo you moow that io the doerense tound ”ou solle to toe toinge io the siaee.”

“i whsh i can _oeed ”hu was in ” said the macc tfrtree. 
“it woutd you know that ”ou cane doeng ” said the macc tfrtree. 
“i toon to then io the sooe,” she macc thehe degan.
“oo you moow th